<a href="https://colab.research.google.com/github/SridharSeshadri56/Decision_Models/blob/main/simpyDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are examples from the simpy documentation with minor edits to help understand the modeling ideas.

In [1]:
pip install simpy

In [2]:
pip install pytest

In [3]:
import simpy
from simpy.util import start_delayed

In [4]:
def car(env):
 while True:
      print('Start parking at %d' % env.now)
      parking_duration = 5
      yield env.timeout(parking_duration)

      print('Start driving at %d' % env.now)
      trip_duration = 2
      yield env.timeout(trip_duration)

In [5]:
env = simpy.Environment()

In [6]:
start_delayed(env, car(env), 4)
env.run(until=15)

Start parking at 4
Start driving at 9
Start parking at 11


In [7]:
env.run(until=20)

Start driving at 16
Start parking at 18


In [8]:
class Car(object):
     def __init__(self, env, delay):
         self.env = env
         # Start the run process everytime an instance is created.
         self.action = start_delayed(env,self.run(), delay)
         print(delay)
     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
             # We yield the process that process() returns
             # to wait for it to finish
             yield self.env.process(self.charge(charge_duration))

             # The charge process has finished and
             # we can start driving again.
             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
         yield self.env.timeout(duration)

In [9]:
env = simpy.Environment()
car = Car(env, 2)     # I am passing delay to start of process run
car1 = Car(env, 3)   # added another car
#car.action = env.process(car.run()) ## here we say process is run
#start_delayed(env, car1.run(),2)           ## here process start is delayed by 2
env.run(until=15)

2
3
Start parking and charging at 2
Start parking and charging at 3
Start driving at 7
Start driving at 8
Start parking and charging at 9
Start parking and charging at 10
Start driving at 14


In [10]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()


In [11]:
class Car(object):
     def __init__(self, env, name):
         self.env = env
         self.name = name   # pass the name of the car to simulation process 
         # Start the run process everytime an instance is created.
       #  self.action = env.process(self.run())

     def run(self):
         while True:
             print(self.name, 'Start parking and charging at %d' % self.env.now)
             charge_duration = 5
             try:
             # We yield the process that process() returns
             # to wait for it to finish
                 yield self.env.process(self.charge(charge_duration))
             except simpy.Interrupt:       # changed to Interrupt
                print(self.name, 'was interrupted. Hope battery full ...')

             # The charge process has finished and
             # we can start driving again.
             print(self.name, 'Start driving at %d' % self.env.now)
             trip_duration = 2
             try:
                 yield self.env.timeout(trip_duration)
             except simpy.Interrupt:
                print(self.name, 'was rudely interrupted again')

     def charge(self, duration):
         yield self.env.timeout(duration)

In [12]:
env = simpy.Environment()
car = Car(env, 3)
car1 = Car(env, 21)
car.action = env.process(car.run())
env.process(driver(env,car))
#env.process(car1.run())
start_delayed(env, car1.run(), 3)
env.run(until=15)
print(env.now)
env.process(driver(env,car))
env.run(until=24)

3 Start parking and charging at 0
3 was interrupted. Hope battery full ...
3 Start driving at 3
21 Start parking and charging at 3
3 Start parking and charging at 5
21 Start driving at 8
21 Start parking and charging at 10
3 Start driving at 10
3 Start parking and charging at 12
15
21 Start driving at 15
21 Start parking and charging at 17
3 Start driving at 17
3 was rudely interrupted again
3 Start parking and charging at 18
21 Start driving at 22
3 Start driving at 23


In [13]:
def car(env, name, bcs, driving_time, charge_duration):
     # Simulate driving to the BCS
     yield env.timeout(driving_time)

     # Request one of its charging spots
     print('%s arriving at %d' % (name, env.now))
     with bcs.request() as req:
         yield req

         # Charge the battery
         print('%s starting to charge at %s' % (name, env.now))
         yield env.timeout(charge_duration)
         print('%s leaving the bcs at %s' % (name, env.now))

In [14]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=3)

In [15]:
for i in range(4):
   env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

In [16]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 2 starting to charge at 4
Car 0 leaving the bcs at 5
Car 3 arriving at 6
Car 3 starting to charge at 6
Car 1 leaving the bcs at 7
Car 2 leaving the bcs at 9
Car 3 leaving the bcs at 11


In [17]:
import simpy
import random

In [18]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))


In [19]:
# Setup and start the simulation
print('Bank renege')
random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


In [20]:
"""
Carwash example.

Covers:

- Waiting for other processes
- Resources: Resource

Scenario:
  A carwash has a limited number of washing machines and defines
  a washing processes that takes some (random) time.

  Car processes arrive at the carwash at a random time. If one washing
  machine is available, they start the washing process and wait for it
  to finish. If not, they wait until they an use one.

"""
import random

import simpy


RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes


class Carwash(object):
    """A carwash has a limited number of machines (``NUM_MACHINES``) to
    clean cars in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        """The washing processes. It takes a ``car`` processes and tries
        to clean it."""
        yield self.env.timeout(WASHTIME)
        print("Carwash removed %d%% of %s's dirt." %
              (random.randint(50, 99), car))


def car(env, name, cw):
    """The car process (each car has a ``name``) arrives at the carwash
    (``cw``) and requests a cleaning machine.

    It then starts the washing process, waits for it to finish and
    leaves to never come back ...

    """
    print('%s arrives at the carwash at %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('%s enters the carwash at %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('%s leaves the carwash at %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    """Create a carwash, a number of initial cars and keep creating cars
    approx. every ``t_inter`` minutes."""
    # Create the carwash
    carwash = Carwash(env, num_machines, washtime)

    # Create 4 initial cars
    for i in range(4):
        env.process(car(env, 'Car %d' % i, carwash))

    # Create more cars while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Car %d' % i, carwash))


# Setup and start the simulation
print('Carwash')
print('Check out http://youtu.be/fXXmeP9TvBg while simulating ... ;-)')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

# Execute!
env.run(until=SIM_TIME)

Carwash
Check out http://youtu.be/fXXmeP9TvBg while simulating ... ;-)
Car 0 arrives at the carwash at 0.00.
Car 1 arrives at the carwash at 0.00.
Car 2 arrives at the carwash at 0.00.
Car 3 arrives at the carwash at 0.00.
Car 0 enters the carwash at 0.00.
Car 1 enters the carwash at 0.00.
Car 4 arrives at the carwash at 5.00.
Carwash removed 97% of Car 0's dirt.
Carwash removed 67% of Car 1's dirt.
Car 0 leaves the carwash at 5.00.
Car 1 leaves the carwash at 5.00.
Car 2 enters the carwash at 5.00.
Car 3 enters the carwash at 5.00.
Car 5 arrives at the carwash at 10.00.
Carwash removed 64% of Car 2's dirt.
Carwash removed 58% of Car 3's dirt.
Car 2 leaves the carwash at 10.00.
Car 3 leaves the carwash at 10.00.
Car 4 enters the carwash at 10.00.
Car 5 enters the carwash at 10.00.
Carwash removed 97% of Car 4's dirt.
Carwash removed 56% of Car 5's dirt.
Car 4 leaves the carwash at 15.00.
Car 5 leaves the carwash at 15.00.
Car 6 arrives at the carwash at 16.00.
Car 6 enters the carwash 